In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
BY_ACCOUNT = dataiku.Dataset("BY_ACCOUNT")
BY_ACCOUNT_df = BY_ACCOUNT.get_dataframe()

df = BY_ACCOUNT_df

In [5]:
customers = df.CUSTOMER.unique()
len(customers)

657099

In [8]:
import string
import nltk
from nltk.tag import pos_tag

df_dict = pd.DataFrame(nltk.corpus.words.words(), columns=['word'])
df_dict['word'] = df_dict['word'].str.upper()

In [0]:
idx = 0

for c in customers:
    
    c_str = c.translate(str.maketrans('', '', string.punctuation))
    f = c_str.split()
    
    print(c_str)
    
    for w in f:
        if (len(w)>1) and (not w.isnumeric()):
            is_in_dict = len(df_dict[df_dict['word']==w])>0
            print(w, is_in_dict)
    
    print()
    idx+=1
    
    if idx>10:
        break;

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

WORD_LIST_df = BY_ACCOUNT_df # For this sample code, simply copy input to output


# Write recipe outputs
WORD_LIST = dataiku.Dataset("WORD_LIST")
WORD_LIST.write_with_schema(WORD_LIST_df)